# Как научить компьютер читать? 

В этой тетрадке мы обучим свой собственный word2vec. Делать мы это будем на каком-нибудь не очень большом тексте, который вам предстоит выбрать самому. На выбор есть [несколько сказок](https://github.com/nevmenandr/word2vec-russian-novels/tree/master/vector-school) и других [литературных штук](https://github.com/nevmenandr/word2vec-russian-novels/tree/master/books_before) из школьной программы. 

In [ ]:
# Ссылка на выбранное вами произведение
# Я взял преступление и наказание (ненвижу Достоевского)
url = 'https://raw.githubusercontent.com/nevmenandr/word2vec-russian-novels/master/books_before/CrimeAndPunishment.txt'


Спарсим текст из файлика.

In [ ]:
import requests

resp = requests.get(url)
text = resp.text 

# Последние 500 символов. Аккуратно! Спойлеры!
print(text[-500:])

## 1. Предобработка

Теперь нам надо его немного предобработать.  Пусть все слова пишутся с маленькой буквы. 

In [ ]:
text = text.lower()

Разобьём весь текст на предложения. 

In [ ]:
from nltk.tokenize import sent_tokenize

sents = sent_tokenize(text)
len(sents)

In [ ]:
sents[220]

Разобьём каждое предложение на отдельные слова.

In [ ]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer('\w+')
tokenizer.tokenize(sents[220])

In [ ]:
# разбейте все предложения на токены 
sents_tokenize  =  ... 

In [ ]:
# Flatten без numpy :) 
words = [item for sent in  sents_tokenize for item in sent]

In [ ]:
len(words) # всего слов

In [ ]:
len(set(words)) # уникальных слов

Можно выбросить все стоп-слова. 

In [ ]:
from nltk.corpus import stopwords

stopwords_ru = stopwords.words('russian') 
stopwords_ru[:10]

In [ ]:
# избавьтесь от стоп-слов 
sents_tokenize = ...

Слов в корпусе не очень много. Давайте лемматизируем их.  В этом нам поможет библиотека **pymorphy2.**

**pymorphy2** — это полноценный морфологический анализатор, целиком написанный на Python. Он также умеет ставить слова в нужную форму (спрягать и склонять). [Документация по pymorphy2.](https://pymorphy2.readthedocs.io/en/latest/)

In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

text = "Филипп пошёл в Авеньон и пленил пап!"
tokens = tokenizer.tokenize(text)

" ".join(morph.normal_forms(token)[0] for token in tokens)

In [ ]:
p = morph.parse('стали')
p

Обработаем все слова из датасета. 

In [ ]:
# лемматизируйте все слова из датасета
sents_tokenize = ...

In [ ]:
# Flatten без numpy :) 
words = [item for sent in  sents_tokenize for item in sent]

In [ ]:
len(words) # всего слов

In [ ]:
len(set(words)) # уникальных слов

Хватит обработок! Мы тут не анализом текстов занимаемся, а нейросетками. Если хочешь больше предобработки, [читай мой мануал](https://nbviewer.jupyter.org/github/FUlyankin/hse_texts_do/blob/master/sem_1/texts_sem1.ipynb) об этом.  Давайте построим словарик с частотностями и перейдём к моделированию. 

In [ ]:
from collections import Counter

word_dict = Counter(words)
word_dict.most_common()[:20]

In [ ]:
words = word_dict.most_common()
len([item for item in words if item[1] >= 3])  # совсем мало :) 

## 2. Моделирование

__Основные параметры:__

* данные должны быть итерируемым объектом 
* `size` — размер вектора, 
* `window` — размер окна наблюдения,
* `min_count` — мин. частотность слова в корпусе,
* `sg` — используемый алгоритм обучения (0 — CBOW, 1 — Skip-gram),
* `sample` — порог для downsampling'a высокочастотных слов,
* `workers` — количество потоков,
* `alpha` — learning rate,
* `iter` — количество итераций,
* `max_vocab_size` — позволяет выставить ограничение по памяти при создании словаря (т.е. если ограничение привышается, то низкочастотные слова будут выбрасываться). Для сравнения: 10 млн слов = 1Гб RAM.

In [ ]:
%%time 
from gensim.models.word2vec import Word2Vec

# size - размерность векторов, которые мы хотим обучить
# window - ширина окна контекста
# min_count - если слово встречается реже, для него не учим модель
model = Word2Vec(size=100, window=2, min_count=3, workers=4)

# строительство словаря, чтобы обучение шло быстрее
model.build_vocab(sents_tokenize)

# обучение модели 
# первый аргумент - наша выборка, генератор будет вкидывать в модель наши тексты, пока они не кончатся
# второй аргумент - число примеров в выборке 
# третий аргумент - количество эпох обучения: сколько раз модель пройдётся по всему корпусу текстов
model.train(sents_tokenize, total_examples=model.corpus_count, epochs=100)

# !NB в ситуации, когда у нас огромный корпус, 100 эпох это слишком много! 

In [ ]:
model.corpus_count # число примеров в обучающей выборке

Смотрим, сколько в модели слов.

In [ ]:
len(model.wv.vocab)

In [ ]:
'старуха' in model.wv.vocab

## 3. Свойства модели

In [ ]:
# вектор слова


In [ ]:
# размерность вектора


In [ ]:
# похожести слов 


In [ ]:
# самые похожие


In [ ]:
# арифметика


## 4. Как дообучить модель? 

Ради чистоты эксперимента сохраним текущую модель и заново подгрузим её. 

In [ ]:
model_path = "./our_w2v.model"
model.save(model_path)

In [ ]:
our_model = Word2Vec.load(model_path)

Подгрузим другое произведение и сделаем для него предобработку. 

In [ ]:
url = 'https://raw.githubusercontent.com/nevmenandr/word2vec-russian-novels/master/vector-school/SkazkaOCareSaltane.txt'

resp = requests.get(url)
text2 = resp.text 

# Последние 500 символов. Аккуратно! Спойлеры!
print(text2[-500:])

Предобработка.

In [ ]:
text2 = text2.lower()
sents2 = sent_tokenize(text2)

sents_tokenize2 = [tokenizer.tokenize(sent) for sent in sents2]
sents_tokenize2 = [[morph.normal_forms(word)[0] for word in text_cur if word not in stopwords_ru]
                      for text_cur in sents_tokenize2]

In [ ]:
sents_tokenize2[10]

In [ ]:
len(sents_tokenize2)

Дополняем модель.

In [ ]:
# обновили словарь
our_model.build_vocab(sents_tokenize2, update=True)

# дообучили модель
our_model.train(sents_tokenize2, total_examples=our_model.corpus_count, epochs=100)

In [ ]:
'ядро' in model.wv.vocab

In [ ]:
'ядро' in our_model.wv.vocab

In [ ]:
our_model.wv.most_similar('ядро')

Пример со старым словом.

In [ ]:
our_model.wv.most_similar('сын')

In [ ]:
model.wv.most_similar('сын')